In [1]:
import requests
import pandas as pd
from mainnet_launch.constants import AUTO_ETH
from web3 import Web3


def paginate_query(api_url: str, query: str, variables: dict, data_col: str) -> list[dict]:
    """
    Helper to page through a GraphQL connection using `first`/`skip`.

    :param api_url: GraphQL endpoint
    :param query: The GraphQL query string, expecting $first and $skip variables
    :param variables: Base variables (e.g. {"autoEthAddress": "..."}).
                      first/skip will be merged in each loop.
    :param batch_size: Number of items to fetch per request.
    :return: List of result dicts
    """

    all_records = []
    skip = 0

    while True:
        vars_with_pagination = {**variables, "first": 500, "skip": skip}
        resp = requests.post(api_url, json={"query": query, "variables": vars_with_pagination})
        resp.raise_for_status()
        batch = resp.json()["data"][data_col]

        if not batch:
            break

        all_records.extend(batch)
        skip += 500

    return all_records


def _get_subgraph_api(chain):
    if chain == "eth":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-eth-mainnet/api"
    elif chain == "base":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-base-mainnet/api"
    else:
        raise ValueError("bad chain", chain)

    return api_url


def fetch_autopool_rebalance_events_from_subgraph(autopool_eth_addr: str, chain: str) -> list[dict]:
    """
    Fetches all AutopoolRebalances entries for the given autopool.
    """
    subgraph_url = _get_subgraph_api(chain)

    query = """
    query($autoEthAddress: String!, $first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: id,
        orderDirection: desc,
        where: { autopool: $autoEthAddress }
      ) {
        transactionHash
        timestamp
        blockNumber

        tokenInAmount
        tokenOut {
        id
        decimals
        
        }        
        destinationInAddress

        tokenIn {
        id
        decimals
        
        }   

        tokenOutAmount
        destinationOutAddress
        
      }
    }
    """

    # Adjust `first` as needed; paginate_query will loop over skip increments of `first`
    df = pd.DataFrame.from_records(
        paginate_query(
            subgraph_url,
            query,
            variables={"autoEthAddress": autopool_eth_addr.lower(), "first": 1000, "skip": 0},
            data_col="autopoolRebalances",
        )
    )

    df["tokenInAddress"] = df["tokenIn"].apply(lambda x: x["id"])
    df["tokenOutAddress"] = df["tokenOut"].apply(lambda x: x["id"])

    df["tokenOutAmount"] = df.apply(
        lambda row: int(row["tokenOutAmount"]) / (10 ** int(row["tokenOut"]["decimals"])), axis=1
    )
    df["tokenInAmount"] = df.apply(
        lambda row: int(row["tokenInAmount"]) / (10 ** int(row["tokenIn"]["decimals"])), axis=1
    )

    df["blockNumber"] = df["blockNumber"].astype(int)

    df["destinationInAddress"] = df["destinationInAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenInAddress"] = df["tokenInAddress"].apply(lambda x: Web3.toChecksumAddress(x))

    df["destinationOutAddress"] = df["destinationOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenOutAddress"] = df["tokenOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))

    return df


rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph("0xa7569A44f348d3D70d8ad5889e50F78E33d80D35", "eth")
rebalance_event_df

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress
0,0xff8915d0bd9054e7678c4366ae8fd5a0a3cce80943be...,1744988327,22296632,148881.150087,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
1,0xff3d22a3564c8121f28d7d47feffb8251785ad2c48e4...,1744997471,22297389,148885.474474,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
2,0xfecf659a8d1c06c45010afc47fcefc6405d906a85e84...,1746235043,22399927,320651.575857,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xC5c95fCad37E466E25e6ecA1977bbF75C0E1004a,{'id': '0xbeef01735c132ada46aa9aa4c54623caa92a...,347222.682150,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xBEEF01735c132Ada46AA9aA4c54623cAA92A64CB,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
3,0xfca3e4ad4e6296b2444ed28a8012cda8694ef5d23ce9...,1746116255,22390138,148075.994800,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,170000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
4,0xfc37985d3c7182369be30f9dbe69409a8b05d008bde2...,1745111003,22306807,437134.294459,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61,{'id': '0x5c20b550819128074fd538edf79791733cce...,500000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x5C20B550819128074FD538Edf79791733ccEdd18,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
...,...,...,...,...,...,...,...,...,...,...,...
294,0x02a5528289efe28718bed54e9c5b0772f16eba69d609...,1744695875,22272381,76347.504659,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,77679.210200,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
295,0x02065d1b16c7c693afe6358f38647b921a095d6876ec...,1745911055,22373171,26611.724622,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,27068.157900,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
296,0x012e26a2633c10388ffc3ee1a2a95c9569b54ff4f918...,1744556327,22260800,140053.102440,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,0xC099899d0278CE83976218Cbe58D01dD382dcA32,{'id': '0x57064f49ad7123c92560882a45518374ad98...,149036.479162,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x57064F49Ad7123C92560882a45518374ad982e85,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85
297,0x00ff23899cd9bb0075ebe5ac23a3faa815088ddd32ab...,1744906595,22289850,148973.430940,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48


In [2]:
from mainnet_launch.database.schema.full import (
    Blocks,
    DestinationTokenValues,
    DestinationStates,
    Destinations,
    AutopoolStates,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)
from mainnet_launch.database.schema.ensure_tables_are_current.update_destinations_states_table import (
    _add_new_destination_states_to_db,
)

from mainnet_launch.database.schema.ensure_tables_are_current.update_autopool_states import (
    _add_new_autopool_states_to_db,
)

_add_new_destination_states_to_db(rebalance_event_df["blockNumber"], AUTO_ETH.chain)
_add_new_autopool_states_to_db(rebalance_event_df["blockNumber"], AUTO_ETH.chain)

destination_states_df = merge_tables_as_df(
    [
        TableSelector(DestinationStates, [DestinationStates.block, DestinationStates.lp_token_spot_price]),
        TableSelector(
            Destinations,
            [Destinations.destination_vault_address, Destinations.underlying_symbol],
            join_on=(DestinationStates.chain_id == DestinationStates.chain_id)
            & (Destinations.destination_vault_address == DestinationStates.destination_vault_address),
        ),
        TableSelector(
            AutopoolStates,
            [AutopoolStates.total_nav],
            join_on=(AutopoolStates.chain_id == DestinationStates.chain_id)
            & (AutopoolStates.block == DestinationStates.block),
        ),
    ],
    where_clause=AutopoolStates.autopool_vault_address == "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35",
)
destination_states_df

2025-05-09 10:43:24.570 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2025-05-09 10:43:25,497 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-09 10:43:25,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-09 10:43:25,687 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-09 10:43:25,688 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-09 10:43:25,878 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-09 10:43:25,879 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-09 10:43:26,077 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-09 10:43:26,077 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destination_states
            WHERE destination_states.chain_id = 1
        
2025-05-09 10:43:26,078 INFO sqlalchemy.engine.Engine [generated in 0.00035s] {}
2025-05-09 10:43:27,676 INFO sqlalchemy.engine.Engine COMMIT
2025-05-09 10:43:27,829 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-09 10:43:27,829 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM auto

,block,lp_token_spot_price,destination_vault_address,underlying_symbol,total_nav
0,20759464,1.036516,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,B-rETH-STABLE,NaN
1,20766617,1.036121,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,B-rETH-STABLE,NaN
2,20773761,1.036456,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,B-rETH-STABLE,NaN
3,20780916,1.036477,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,B-rETH-STABLE,NaN
4,20788084,1.036502,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,B-rETH-STABLE,NaN
...,...,...,...,...,...
35805,22272381,1.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,autoUSD,1.010120e+07
35806,22373171,1.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,autoUSD,2.429946e+07
35807,22260800,1.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,autoUSD,8.023007e+06
35808,22289850,1.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,autoUSD,1.348924e+07


In [6]:
# this works

In [31]:
price_df = destination_states_df[["block", "destination_vault_address", "lp_token_spot_price"]]

df1 = rebalance_event_df.merge(
    price_df,
    left_on=["blockNumber", "destinationInAddress"],
    right_on=["block", "destination_vault_address"],
    how="left",
)
# rename & drop the merge‑helper cols
df1.rename(columns={"lp_token_spot_price": "lp_token_in_spot_price"}, inplace=True)
df1.drop(columns=["block", "destination_vault_address"], inplace=True)


df1 = df1.merge(
    price_df,
    left_on=["blockNumber", "destinationOutAddress"],
    right_on=["block", "destination_vault_address"],
    how="left",
)
df1.rename(columns={"lp_token_spot_price": "lp_token_out_spot_price"}, inplace=True)
df1.drop(columns=["block", "destination_vault_address"], inplace=True)

df1["spot_value_in"] = df1["tokenInAmount"] * df1["lp_token_in_spot_price"]
df1["spot_value_out"] = df1["tokenOutAmount"] * df1["lp_token_out_spot_price"]


df1["swap_cost"] = df1["spot_value_out"] - df1["spot_value_in"]
df1["slippage"] = (100 * (df1["spot_value_out"] - df1["spot_value_in"])) / df1["spot_value_out"]

block_and_nav = destination_states_df[["block", "total_nav"]].drop_duplicates()
df1 = pd.merge(df1, block_and_nav, left_on="blockNumber", right_on=["block"])
df1.index = pd.to_datetime(df1["timestamp"].astype(int), unit="s", utc=True)
df1 = df1.sort_index()
df1.to_csv("autoUSD_rebalance_slippage.csv")

In [28]:
df1.tail()

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress,lp_token_in_spot_price,lp_token_out_spot_price,spot_value_in,spot_value_out,swap_cost,slippage,block,total_nav
timestamp,,,,,,,,,,,,,,,,,,,
2025-04-15 05:44:35+00:00,0x02a5528289efe28718bed54e9c5b0772f16eba69d609...,1744695875,22272381,76347.504659,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,77679.210200,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017084,1.000000,77651.825428,77679.210200,27.384772,0.035254,22272381,1.010120e+07
2025-04-29 07:17:35+00:00,0x02065d1b16c7c693afe6358f38647b921a095d6876ec...,1745911055,22373171,26611.724622,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,27068.157900,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017144,1.000000,27067.956028,27068.157900,0.201872,0.000746,22373171,2.429946e+07
2025-04-13 14:58:47+00:00,0x012e26a2633c10388ffc3ee1a2a95c9569b54ff4f918...,1744556327,22260800,140053.102440,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,0xC099899d0278CE83976218Cbe58D01dD382dcA32,{'id': '0x57064f49ad7123c92560882a45518374ad98...,149036.479162,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x57064F49Ad7123C92560882a45518374ad982e85,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,1.071202,1.006465,150025.163440,150000.000000,-25.163440,-0.016776,22260800,8.023007e+06
2025-04-17 16:16:35+00:00,0x00ff23899cd9bb0075ebe5ac23a3faa815088ddd32ab...,1744906595,22289850,148973.430940,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.006858,1.000000,149995.090730,150000.000000,4.909270,0.003273,22289850,1.348924e+07
2025-04-15 00:55:47+00:00,0x0000f50368b9d54cf50696552d1adeb3d070298ee8a7...,1744678547,22270944,196495.452853,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,199999.999999,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,1.017086,1.000000,199852.774160,199999.999999,147.225839,0.073613,22270944,1.003466e+07


In [18]:
df1.isna().sum()

transactionHash            0
timestamp                  0
blockNumber                0
tokenInAmount              0
tokenOut                   0
destinationInAddress       0
tokenIn                    0
tokenOutAmount             0
destinationOutAddress      0
tokenInAddress             0
tokenOutAddress            0
lp_token_in_spot_price     0
lp_token_out_spot_price    1
spot_value_in              0
spot_value_out             1
swap_cost                  1
slippage                   1
block                      0
total_nav                  0
dtype: int64

In [33]:
import plotly.express as px

px.bar(df1.resample("1d")[["swap_cost"]].sum(), y="swap_cost", title="Swap Cost")

In [8]:
avg_nav = df1.resample("1d")["total_nav"].mean()
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
total_swap_cost

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [ ]:
avg_nav = df1.resample("1d")["total_nav"].mean() * 1e12
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
import plotly.express as px

percent_of_swap_cost_per_day = 100 * (total_swap_cost) / avg_nav
px.bar(percent_of_swap_cost_per_day)

In [ ]:
df1.tail(1).T

In [ ]:
df1.index = pd.to_datetime(df1["timestamp"].astype(int), unit="s", utc=True)

df1.sort_index().to_csv("autoUSD_slippage_with_nav.csv")

In [ ]:

import plotly .express as px



df1.sort_values(')

In [ ]:
df1[
    [
        "transactionHash",
        "timestamp",
        "blockNumber",
        "tokenInAmount",
        "destinationInAddress",
        "tokenOutAmount",
        "destinationOutAddress",
        "tokenInAddress",
        "tokenOutAddress",
        "lp_token_in_spot_price",
        "lp_token_out_spot_price",
        "spot_value_in",
        "spot_value_out",
        "slippage",
    ]
].to_csv("autoUSD_slippage.csv")

In [ ]:
df1["slippage"] < 0

In [ ]:
df1["slippage"]

In [ ]:
px.histogram(df1["swap_cost"], title="autoUSD slippage")

In [ ]:
destination_states_df

In [ ]:
destination_states_df[destination_states_df["underlying"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"]

In [ ]:
df1["tokenOutAddress"].value_counts()

In [ ]:
destination_states_df

In [ ]:
rebalance_event_df

In [ ]:
rebalance_event_df[["blockNumber", "destinationInAddress", "tokenInAddress"]]

In [ ]:
destination_states_df

In [ ]:
destination_states_df[["block", "destination_vault_address", "underlying"]]

In [ ]:
destination_states_df = destination_states_df.drop_duplicates()

In [ ]:
rebalance_event_df.head(1).T

In [ ]:
# 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48

In [ ]:
token_value_df[
    (token_value_df["block"] == 22296632)
    & (token_value_df["destination_vault_address"] == "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35")
    & (token_value_df["token_address"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48")
]

In [ ]:
rebalance_event_df

In [ ]:
destination_states_df["underlying_symbol"].value_counts()